# Demand Generation from Flight Departures
Author Ilias Parmaksizoglou

A Jupyter notebook developed to transform historical data from Terminal 1 of Malpensa International Airport to trip starts associated with a specific time-window throughout the day and a flight departure. 



In [1]:
# Necessary Python libraries
import pandas as pd
import numpy as np
import datetime
import random
import geopandas as gpd
import calendar

import os
os.environ['USE_PYGEOS'] = '0'
from demand_generation import *

# Get the current notebook's directory
notebook_dir = os.path.abspath(os.path.curdir)

# Construct the path to the parent directory
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))




c:\Users\iparm\anaconda3\envs\mesa\Lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\iparm\AppData\Local\Temp\ipykernel_34764\3860749697.py:6: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However

First we load the necessary data relating to departures. Data are provided from SEA Aeroporti Milano and relate to the period between 1/6/2022 and 30/6/2022. The provided data contain both departing and arriving flights, but we are only interested in departures for now. The datasets loaded to the notebooks are: 

* mxp_data.csv (located on the data/mxp directory) - Containing arrivals/departures from MXP
* airlines.xlsx (located on the data/mxp directory) - Containing ICAO codes for associated airlines

Furthermore we define, time-window length of reference at 30 minutes. Additionally, we define factor trip_start, signalling the number of clusters in advance of their flight that most people start the trip to the airport.

In [2]:
# Initialize Inputs
dir = os.path.join(parent_dir,"data\\mxp")
data = pd.read_csv(os.path.join(dir,'mxp_data.csv')) 
data_np = data.values # Converting pandas Dataframe to numpy
airlines = pd.read_excel(os.path.join(dir,'airlines.xlsx'))

# Clustering globals
period = 30
trip_start = 210//period
tot_periods = int(1440//period)

# Date related
year = 2023 # Correct is 2022 we just assume that traffic in 2023 will be similar
month = 6
month_name = calendar.month_name[month] 

First we want to isolate the flights that are of interest to us, as we only deal with departures. Additionally, there are some formatting related issues that are taken care of in this step. In the end we create a departures plan, containing information for all the flights that we are interested on. Stored information contain:

* Departure Time in minutes from midnight
* Departure Cluster - Time window that the flight departs
* Departure Mu - Time window that the average user will start their trip towards the airport for associate flight
* Flight Code - Generated identifier for each flight
* Date - Date of flight departure
* Pax - Passengers associated to that flight
* Company - Airline operating that flight

In [3]:
# Initializing empty dictionary to store information
departure_mod = {'idxs' : [],'Departure Time (min)': [], 'Departure Cluster': [], 'Departure Mu' : [], 'Flight Code' : [], "SCHENGEN" : [], "TYPE" : [], 'NATIONAL' : []}
arrival_mod = {'idxs' : [],'Arrival Time (min)': [], 'Arrival Cluster': [], 'Flight Code' : []}

# Start iterating across all flights
for idx,row in enumerate(data_np):

   
    # Departure time is written as string in the 00:00 format but without ':'
    timestamp = str(row[0])
    
    # Minutes are the last two digits of the timestamp regardless
    minutes = int(timestamp[-2:])
    
    # Hours are zero if length of string smaller-equal to two
    hours = 0 
    if len(timestamp) >2:
        hours = int(timestamp[0:-2])

    # Departure from midnight in minutes and clusters
    flight_arr_dep = 60*hours + minutes
    cluster = flight_arr_dep//period

    # Examined day without year and month - row[1] contains datetime
    day = datetime.datetime.strptime(row[1], '%d/%m/%Y').date().day

    # Construct specialized code for flight based on Airline|Day|Departure Time|Random ID  
    flight_code=airlines.loc[airlines['Airline'] == row[4]].values[0][1]+'|'+str(day)+'|'+str(flight_arr_dep)+'|'+row[3]+str(idx%100)
    schengen = (airlines[airlines['Airline'] == row[4]]['SCHENGEN']).values[0]
    type_f = (airlines[airlines['Airline'] == row[4]]['TYPE']).values[0]
    national = False
    if schengen == 1:
        schengen = False
    else:
        val = random.random()
        if val <0.15:
            schengen = False
        else:
            schengen = True
            if type_f == 'LEISURE':
                val = random.random()
                if val <0.5:
                    schengen = False
                else:
                    if random.random() <0.8:
                        national = True

            if type_f  == 'LC':
                if random.random() <0.5:
                    national = True 

            if type_f  == 'LEG':
                if random.random() <0.01:
                    national = True  
    # Omit Departures - row[3] associated with flight type
    if row[3] == "A":
        # Save only Departures to the dictionary
        arrival_mod['idxs'].append(idx)
        arrival_mod['Arrival Time (min)'].append(flight_arr_dep)
        arrival_mod['Arrival Cluster'].append(cluster)
        arrival_mod['Flight Code'].append(flight_code)
    else:         
        # Associate departure time with trip_start factor - Insert gross assumption when flight leads to the previous day
        mu = cluster-trip_start
        if type_f != "LEG" or national == True:
            mu+=1
        if not(schengen):
            mu-=2

        if mu < 0:
            mu = 0 # We could use (mu = mu + 60*24/period) but then Date of trip start and Flight would differ - potentially problematic

        # Save only Departures to the dictionary
        departure_mod['idxs'].append(idx)
        departure_mod['Departure Time (min)'].append(flight_arr_dep)
        departure_mod['Departure Cluster'].append(cluster)
        departure_mod['Departure Mu'].append(int(mu))
        departure_mod['Flight Code'].append(flight_code)
        departure_mod['SCHENGEN'].append(schengen)
        departure_mod['TYPE'].append(type_f)
        departure_mod['NATIONAL'].append(national)

# Create Dataframe from dictionary with idxs as index
departures = pd.DataFrame(departure_mod)
departures.set_index('idxs',inplace=True)
departures.index.name = None

arrivals = pd.DataFrame(arrival_mod)
arrivals.set_index('idxs',inplace=True)
arrivals.index.name = None

# Merge Dataframe with loaded plan, sort by Date and remove unnecessary information
departures_plan = pd.concat([departures, data.loc[departure_mod['idxs']]], axis=1, join='inner')
departures_plan = departures_plan.sort_values(["Date",'Time'])
departures_plan['Date'] = departures_plan['Date'].str.replace("2022","2023").str.replace("06","6") # Apply year 'correction' and month correction
departures_plan = departures_plan.drop(['Type','Time'],axis=1)
departures_plan.reset_index(inplace=False)

arrivals_plan = pd.concat([arrivals, data.loc[arrival_mod['idxs']]], axis=1, join='inner')
arrivals_plan = arrivals_plan.sort_values(["Date",'Time'])
arrivals_plan['Date'] = arrivals_plan['Date'].str.replace("2022","2023").str.replace("06","6") # Apply year 'correction' and month correction
arrivals_plan = arrivals_plan.drop(['Type','Time'],axis=1)
arrivals_plan.reset_index(inplace=False)

# Save Departure plan to MXP folder
path = f"{dir}\\{month_name}"
exists = os.path.exists(path)

# Create a new directory because it does not exist
if not exists:
    os.makedirs(path)

departures_plan.to_csv(f"{path}\\departures_plan.csv",index=False)

# df_category_A = departures_plan[departures_plan['SCHENGEN'] == True]
# df_category_A = df_category_A[df_category_A['TYPE'] == 'LEISURE']
# df_category_A_I = df_category_A[df_category_A['NATIONAL'] == True]
# df_category_A_N = df_category_A[df_category_A['NATIONAL'] == False]

# df_category_B = departures_plan[departures_plan['SCHENGEN'] == False]
# df_category_B = df_category_B[df_category_B['TYPE'] == 'LEISURE']

# df_category_C = departures_plan[departures_plan['SCHENGEN'] == True]
# df_category_C = df_category_C[df_category_C['TYPE'] == 'LEG']
# df_category_C_I = df_category_C[df_category_C['NATIONAL'] == True]
# df_category_C_N = df_category_C[df_category_C['NATIONAL'] == False]

# df_category_D = departures_plan[departures_plan['SCHENGEN'] == False]
# df_category_D = df_category_D[df_category_D['TYPE'] == 'LEG']

# df_category_E = departures_plan[departures_plan['SCHENGEN'] == True]
# df_category_E = df_category_E[df_category_E['TYPE'] == 'LC']
# df_category_E_I = df_category_E[df_category_E['NATIONAL'] == True]
# df_category_E_N = df_category_E[df_category_E['NATIONAL'] == False]

# df_category_F = departures_plan[departures_plan['SCHENGEN'] == False]
# df_category_F = df_category_F[df_category_F['TYPE'] == 'LC']

# # Calculate the sum of the 'Value' column for each DataFrame
# sum_A = df_category_A['Pax'].sum()
# sum_B = df_category_B['Pax'].sum()
# sum_C = df_category_C['Pax'].sum()
# sum_D = df_category_D['Pax'].sum()
# sum_E = df_category_E['Pax'].sum()
# sum_F = df_category_F['Pax'].sum()

# sum_A_I = df_category_A_I['Pax'].sum()
# sum_A_N = df_category_A_N['Pax'].sum()
# sum_C_I = df_category_C_I['Pax'].sum()
# sum_C_N = df_category_C_N['Pax'].sum()
# sum_E_I = df_category_E_I['Pax'].sum()
# sum_E_N = df_category_E_N['Pax'].sum()

# print(sum_A/(sum_A+sum_B+sum_C+sum_D+sum_E+sum_F),sum_B/(sum_A+sum_B+sum_C+sum_D+sum_E+sum_F))
# print(sum_C/(sum_A+sum_B+sum_C+sum_D+sum_E+sum_F),sum_D/(sum_A+sum_B+sum_C+sum_D+sum_E+sum_F))
# print(sum_E/(sum_A+sum_B+sum_C+sum_D+sum_E+sum_F),sum_F/(sum_A+sum_B+sum_C+sum_D+sum_E+sum_F))

# print(sum_A_I/(sum_A+sum_B+sum_C+sum_D+sum_E+sum_F),sum_A_N/(sum_A+sum_B+sum_C+sum_D+sum_E+sum_F))
# print(sum_C_I/(sum_A+sum_B+sum_C+sum_D+sum_E+sum_F),sum_C_N/(sum_A+sum_B+sum_C+sum_D+sum_E+sum_F))
# print(sum_E_I/(sum_A+sum_B+sum_C+sum_D+sum_E+sum_F),sum_E_N/(sum_A+sum_B+sum_C+sum_D+sum_E+sum_F))

arrivals_plan.to_csv(f"{path}\\arrivals_plan.csv",index=False)
total_flights = pd.concat([departures_plan,arrivals_plan],axis=0)
total_flights.to_csv(f"{path}\\total_flights_plan.csv",index=False)

Now we have to isolate the departures per day of the examined month. We will individual plans for all the days of the examined month and save them within that directory. This will be the building blocks to create the window trip start to flight matrix that estimates incoming demand per time-window for all flights within the examined day. To create that demand we will essentially sample from a normal distribution centered on the associated $\mu$ of each flight, and a variance of a single cluster. Finally we split the traffic to Milano Metropolitan Area originating and rest of the traffic. Based on reported data from SEA, the modal share of Milano is roughly 50 % 

In [ ]:
def weighted_choice(percent=50):
    return random.randrange(100) < percent

# Get number of days within examined month
num_days = calendar.monthrange(year, month)[1]
no_gates = 72
gates = list(range(1,no_gates+1))
# Align days with single digit format on dataset
days = [datetime.date(year, month, day).strftime('X%d/X%m/%Y').replace('X0','X').replace('X','') for day in range(1, 11)]

for day in days:

    # Isolate Plan for examined day
    day_plan = total_flights[total_flights["Date"]==day]
    day_plan = day_plan.reset_index(drop=True)
    day_plan = day_plan.sort_values(by=['Departure Time (min)','Arrival Time (min)', "SCHENGEN"])

    assigned_gates = []
    passport_needed = []
    for idx, f in day_plan.iterrows():
        gate = random.choice(gates)

        assigned_gates.append(gate)
        if gate not in range(1,25):
            gates.remove(gate)
        
        if gates == range(1,25):
            gates = list(range(1,no_gates+1))

        if not(f[-1]): 
            passport_needed.append(1)
        else:
            passport_needed.append(0)
    
    day_plan["Gate"] = assigned_gates
    day_plan["Passport"] = passport_needed

    # Define path of saved day plane
    path = f"{dir}\\{month_name}\\{day.replace('/','-')}"
    exists = os.path.exists(path)

    # Create a new directory because it does not exist
    if not exists:
        os.makedirs(path)
    
    # Save file to csv
    departures_plan = day_plan.drop(columns=['Arrival Time (min)','Arrival Cluster'])   
    departures_plan =departures_plan.dropna(axis=0)
    departures_plan = departures_plan.reset_index(drop=False)
    departures_plan = departures_plan[['Flight Code','Date','Company','Pax','Gate','Passport','Departure Time (min)','Departure Cluster','Departure Mu','index', 'SCHENGEN', 'TYPE', 'NATIONAL']]
    departures_plan.to_csv(f"{path}//departures_plan.csv",index=False)

    arrivals_plan = day_plan.drop(columns=['Departure Time (min)','Departure Cluster','Departure Mu'])
    arrivals_plan =arrivals_plan.dropna(axis=0)
    arrivals_plan = arrivals_plan.reset_index(drop=False)
    arrivals_plan = arrivals_plan[['Flight Code','Date','Company','Pax','Gate','Passport','Arrival Time (min)','Arrival Cluster','index']]   
    arrivals_plan.to_csv(f"{path}//arrivals_plan.csv",index=False)

    # We define the number of passengers belonging to a departure mu across all flights
    incoming = departures_plan.groupby(["Departure Mu"])["Pax"].sum()
    outgoing = arrivals_plan.groupby(["Arrival Cluster"])["Pax"].sum()
    
    flight_to_arrival= np.zeros((arrivals_plan.shape[0],tot_periods))

    for idx, r in arrivals_plan.iterrows():
        flight_to_arrival[idx][int(r[-2])]= r[4] # r[-2] is arrival cluster

    # Save distribution to trip matrix
    arrival_matrix = pd.DataFrame(data = flight_to_arrival, 
                    index = arrivals_plan['Flight Code'].tolist(), 
                    columns = list(range(1,tot_periods+1)))

    # Assigned demand to a flight based on its traffic
    flight_to_departure = np.zeros((departures_plan.shape[0],tot_periods))

    # Loop over all time windows of the day in incoming
    for mu,value in incoming.items():

        # Define distribution of passengers around that mu
        incoming_distribution = np.random.normal(mu, 1, value).round().astype(int)

        # Isolate flights that share that mu
        flights_mu = departures_plan[departures_plan["Departure Mu"]==mu]
        pax = flights_mu.values[:,3] # Number of pax by r[3]


        for j in list(set(incoming_distribution)):
                      
            # Random assignment of passengers to time-window j based on passengers on flight
            choices = random.choices(flights_mu.index.tolist(), weights=tuple(pax), k=np.count_nonzero(incoming_distribution == j))

            # Correction based on selected choices being sampled less times than they should
            for idx2,id in enumerate(flights_mu.index.tolist()):
                id2 = [x for x in choices if x == id]
                for x in range(len(id2)):
                    if sum(flight_to_departure[id,:]) < pax[idx2]: 
                        flight_to_departure[id,j] +=1
                    else:
                        pax[idx2] = 0

    # Correction for flights that were sampled more time than they should
    for idx2,id in enumerate(departures_plan.index.tolist()):                
        if departures_plan['Pax'][idx2]>sum(flight_to_departure[id,:]):
            mu = int(departures_plan['Departure Mu'][idx2])
            flight_to_departure[id][mu-1]+= departures_plan['Pax'][idx2]-sum(flight_to_departure[id,:])

    # Save distribution to trip matrix
    departure_matrix = pd.DataFrame(data = flight_to_departure, 
                    index = departures_plan['Flight Code'].tolist(), 
                    columns = list(range(1,tot_periods+1)))
    
    # Apply Checks
    check = departures_plan[['Pax','Flight Code']]
    check = check.set_index('Flight Code')
    check.index.name = None
    check2 = departure_matrix.sum(axis=1, numeric_only=True)
    check = check.values.reshape(check.values.shape[1],check.values.shape[0])[0]
    assert check.any() == check2.values.any(), "Distribution does not match reported occupancy"
    departure_matrix.to_csv(f"{path}//departures_matrix.csv")
    arrival_matrix.to_csv(f"{path}//arrivals_matrix.csv")

We can now continue by assigning this passengers to specific origins within Milano. First we will create demand per time-window that is region specific based on the trip matrices already developed and the we will define a direct point, number of passengers and activation time for these trips. The loop below generates all needed files for the Month of June. However, saving these files is not necessary within the simulation.   

In [ ]:
# pop_data = pd.read_excel(f"{parent_dir}/data/milano/milano_population_data.xlsx")
# varese_pop = pd.read_excel(f"{parent_dir}/data/milano/varese_population_data.xlsx")
# polygons = gpd.read_file(f'{parent_dir}/data/milano/nils_milano.geojson')
# trip_persons = [1,2,3,4,5]

# for day in days:
#     departure_flight_data = pd.read_csv(f"{parent_dir}/data/mxp/{month_name}/{day.replace('/','-')}/departures_plan.csv")
#     departure_data = pd.read_csv(f"{parent_dir}/data/mxp/{month_name}/{day.replace('/','-')}/departures_matrix.csv",index_col=0)

#     for cluster in range(1,tot_periods+1):
#         print(cluster)
#         demand = generate_passenger_demand(pop_data,departure_data,cluster,day.replace('/','-'),varese_pop,save=True)
#         pass_dist_departure = demand_to_flight(day.replace('/','-'),cluster,demand,departure_data,polygons,varese_pop,departure_flight_data,trip_persons,save=True)
#     break
#     # arrivals_flight_data = pd.read_csv(f"{parent_dir}/data/mxp/{month_name}/{day.replace('/','-')}/arrivals_plan.csv")
#     # arrivals_data = pd.read_csv(f"{parent_dir}/data/mxp/{month_name}/{day.replace('/','-')}/arrivals_matrix.csv",index_col=0)

#     # for cluster in range(1,tot_periods+1):
#     #     demand = generate_passenger_demand(pop_data,arrivals_data,cluster,day.replace('/','-'),varese_pop,flag='arrivals',save=True)
#     #     pass_dist_arrival = demand_to_flight(day.replace('/','-'),cluster,demand,arrivals_data,polygons,varese_pop,arrivals_flight_data,trip_persons,flag='arrivals',save=True)

